In [1]:
import tensorflow as tf
import tensorflow_io as tfio
import os 



/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

In [112]:
def convert_to_spectogram(wave):
     # Create spectogram 
    # 1. Fast fourier transform 
    spectrogram = tf.signal.stft(wave, frame_length=256, frame_step=128,  window_fn=tf.signal.inverse_stft_window_fn(frame_step))  # Paper: 'Automated detection of gunshots in tropical forests using CNN' 
    # frame_length =  window length in samples
    # frame_step = number of samples to step
    # 'Time frequency compromise' 
    # if window size is small: you get good time resolution in exchange of poor frequency resolution 

    # 2. Obtain the magnitude of the STFT
    # spectrogram = tf.abs(spectrogram)

    # 3. Tranform it into appropiate format for deep learning model by adding the channel dimension (in this case 1)
    # spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [99]:
def read_in_data(file_name, label): 

    file_contents = tf.io.read_file(file_name) #retuns a string 
    wave, _ = tf.audio.decode_wav(file_contents, desired_channels=1) # transforms string into actual wav
    wave = wave - tf.reduce_mean(wave) # remove the mean 
    wave = wave / tf.reduce_max(tf.abs(wave)) #normalize 
    wave = tf.squeeze(wave, axis= -1) #removes axis 
    return wave, label

In [3]:
file = '/Users/rosameliacarioni/University/Thesis/code/data/train/Clips_ELP/ecoguns0.wav'


In [113]:
wave, label = read_in_data(file, 1)
spect= convert_to_spectogram(wave)

In [114]:
wave

<tf.Tensor: shape=(7644,), dtype=float32, numpy=
array([-0.08867513,  0.00858117,  0.21996477, ...,  0.04728521,
        0.14057188, -0.04004698], dtype=float32)>

In [116]:
spect

<tf.Tensor: shape=(58, 129), dtype=complex64, numpy=
array([[-1.09455156e+01 +0.j        ,  2.51917267e+00-10.109248j  ,
         6.69474173e+00 +0.9809136j , ..., -9.44817066e-02 -0.03853589j,
         9.67510939e-02 -0.11757708j,  1.90820694e-02 +0.j        ],
       [-4.18903999e+01 +0.j        ,  2.57818909e+01 -0.5289569j ,
         9.66362298e-01 +7.1473694j , ..., -9.55257714e-02 -0.46248984j,
         1.50493622e-01 +0.40819922j, -1.00242615e-01 +0.j        ],
       [ 2.50697556e+01 +0.j        , -1.84537449e+01+17.603657j  ,
         2.16696405e+00 -0.64258325j, ..., -6.08183146e-02 +0.10573232j,
        -2.20406532e-01 -0.20995998j,  2.49312401e-01 +0.j        ],
       ...,
       [-5.16371822e+00 +0.j        ,  5.10531235e+00+13.309803j  ,
        -6.18728638e-01 -7.0034447j , ...,  3.86216164e-01 +0.22757936j,
        -6.65707588e-02 -0.4538269j , -1.19612217e-02 +0.j        ],
       [ 1.68410931e+01 +0.j        , -7.85362911e+00 -5.601885j  ,
        -2.88678980e+00 +7.

In [57]:
#spectrogram = tf.complex(spect, tf.zeros_like(spect))  # convert to complex

In [117]:
frame_length=256
frame_step=128

waveform = tf.random.normal(dtype=tf.float32, shape=[1000])
stft = tf.signal.stft(waveform, frame_length, frame_step)
inverse_stft = tf.signal.inverse_stft(
    stft, frame_length, frame_step,
    window_fn=tf.signal.inverse_stft_window_fn(frame_step))

In [ ]:
# Convert the magnitude spectrogram to a complex-valued tensor
import numpy as np
stft = spect * tf.exp(tf.complex(0.0, 2.0 * np.pi * np.random.rand(*spect.shape)))

# Compute the iSTFT
frame_length = 256
frame_step = 128
waveform = tf.signal.inverse_stft(stft, frame_length=frame_length, frame_step=frame_step)



In [118]:
frame_length=256
frame_step=128

waveform = tf.signal.inverse_stft(spect, frame_length, frame_step, window_fn=tf.signal.inverse_stft_window_fn(frame_step))
#waveform = tf.signal.inverse_stft(spect, frame_length, frame_step)



In [119]:
waveform

<tf.Tensor: shape=(7552,), dtype=float32, numpy=
array([ 0.0000000e+00,  2.0201928e-10,  7.9947732e-08, ...,
       -1.1397610e-07,  9.0696535e-09,  3.4792208e-10], dtype=float32)>